In [308]:
import geopandas as gpd
import pickle
from sklearn.neighbors import BallTree
import numpy as np
from shapely.geometry import LineString
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import compress
import pandas as pd

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
def load_obj(name ):
    with open(root_path + 'WorkingData/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
def save_obj(name , obj, extra=""):
    with open(root_path + 'WorkingData/' + extra + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#Import map of Euston for tests
whichmap = "Milford Haven"

Mapset = load_obj("Clipped5k_"+whichmap)

Mapset = load_obj("All_Maps")

In [ ]:
def distance(src_points, candidates, source_name, out_name, keep_d_n=[True,True]):
    
    dist_name = "Distance_To_Nearest_" + out_name + "(m)"
    name_name = "Name_of_Nearest_" + out_name
    
    candidates = candidates.reset_index(drop=True)
    
    dist_df = src_points.geometry.apply(lambda g: candidates.distance(g))
    min_distance = list(dist_df.min(axis=1).round(0))
    min_name = list(candidates[source_name].iloc[dist_df.idxmin(axis=1)])
    
    df = pd.DataFrame(list(zip(min_distance, min_name)),
                      columns =[dist_name, name_name],
                     index =src_points.index)
    
    return df

In [ ]:
Mapset['NSPL_gdf'].join(distance(Mapset['NSPL_gdf'], Mapset["A_Roads"], "roadNumber", "A_Road"))